In [1]:
!pip install unsloth
# to get latetst ulsolth packaages
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179

Found existing installation: unsloth 2024.12.12
Uninstalling unsloth-2024.12.12:
  Successfully uninstalled unsloth-2024.12.12
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-dbdfi88y
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-dbdfi88y
  Resolved https://github.com/unslothai/unsloth.git to commit 87f5bffc45a8af7f23a41650b30858e097b86418
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.12.12-py3-none-any.whl size=175166 sha256=b5a34e8e0e5b6c869cf4ad93d8b80922673b8912b94d5de9115bcd1556aaa0b0
  Stored in directory: /tmp/pip-ephem-wheel-cache-7w0n5qd5/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # auto rope scaling
dtype = None
load_in_4bit = True #4bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.12: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # 0 is optimized
    bias = "none",    #  is optimized

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,  # = stabilized LoRA
    loftq_config = None, # Low-Rank Optimal Fine-Tuning Quantization.
)

Unsloth 2024.12.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
import pandas as pd
from datasets import Dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

# Load the dataset
file_path = "Bhagwad_Gita.csv"  # Path datset

#  data conversations
def create_conversations(row):
    return [
        {"from": "user", "value": f"Shloka: {row['Shloka']} Transliteration: {row['Transliteration']}"},
        {"from": "assistant", "value": f"English Meaning: {row['EngMeaning']} Hindi Meaning: {row['HinMeaning']} Word Meaning: {row['WordMeaning']}"}
    ]

df["conversations"] = df.apply(create_conversations, axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Apply the Unsloth template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1"
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Standardize and format the dataset
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched=True)

# Save the processed dataset
dataset.save_to_disk("Bhagwad_Gita_prepared")


Standardizing format:   0%|          | 0/701 [00:00<?, ? examples/s]

Map:   0%|          | 0/701 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/701 [00:00<?, ? examples/s]

In [7]:
dataset[5]["conversations"]

[{'content': 'Shloka: युधामन्युश्च विक्रान्त उत्तमौजाश्च वीर्यवान् |\nसौभद्रो द्रौपदेयाश्च सर्व एव महारथाः ||१-६|| Transliteration: yudhāmanyuśca vikrānta uttamaujāśca vīryavān .\nsaubhadro draupadeyāśca sarva eva mahārathāḥ ||1-6||',
  'role': 'user'},
 {'content': 'English Meaning: 1.6. "The strong Yodhamanyu and the brave Uttamaujas, the son\nof Subhadra (Abhimanyu, the son of Subhadra and Arjuna), and the sons of\nDraupadi, all of great chariots (great heroes). Hindi Meaning: ।।1.6।।पराक्रमी युधामन्यु,\xa0 बलवान् उत्तमौजा,\xa0 सुभद्रापुत्र (अभिमन्यु) और द्रोपदी के पुत्र -- ये सब महारथी हैं। Word Meaning: 1.6 युधामन्युः Yudhamanyu? च and? विक्रान्तः the strong? उत्तमौजाः Uttamaujas? च and? वीर्यवान् the brave? सौभद्रः the son of Subhadra? द्रौपदेयाः the sons of Draupadi? च and? सर्वे all? एव even? महारथाः great carwarriors.No Commentary.',
  'role': 'assistant'}]

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Map (num_proc=2):   0%|          | 0/701 [00:00<?, ? examples/s]

In [9]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/701 [00:00<?, ? examples/s]

In [10]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nShloka: युधामन्युश्च विक्रान्त उत्तमौजाश्च वीर्यवान् |\nसौभद्रो द्रौपदेयाश्च सर्व एव महारथाः ||१-६|| Transliteration: yudhāmanyuśca vikrānta uttamaujāśca vīryavān.\nsaubhadro draupadeyāśca sarva eva mahārathāḥ ||1-6||<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nEnglish Meaning: 1.6. "The strong Yodhamanyu and the brave Uttamaujas, the son\nof Subhadra (Abhimanyu, the son of Subhadra and Arjuna), and the sons of\nDraupadi, all of great chariots (great heroes). Hindi Meaning: ।।1.6।।पराक्रमी युधामन्यु,\xa0 बलवान् उत्तमौजा,\xa0 सुभद्रापुत्र (अभिमन्यु) और द्रोपदी के पुत्र -- ये सब महारथी हैं। Word Meaning: 1.6 युधामन्युः Yudhamanyu? च and? विक्रान्तः the strong? उत्तमौजाः Uttamaujas? च and? वीर्यवान् the brave? सौभद्रः the son of Subhadra? द्रौपदेयाः the sons of Draupadi? च and? सर्वे all? एव e

In [11]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                  \n\nEnglish Meaning: 1.6. "The strong Yodhamanyu and the brave Uttamaujas, the son\nof Subhadra (Abhimanyu, the son of Subhadra and Arjuna), and the sons of\nDraupadi, all of great chariots (great heroes). Hindi Meaning: ।।1.6।।पराक्रमी युधामन्यु,\xa0 बलवान् उत्तमौजा,\xa0 सुभद्रापुत्र (अभिमन्यु) और द्रोपदी के पुत्र -- ये सब महारथी हैं। Word Meaning: 1.6 युधामन्युः Yudhamanyu? च and? विक्रान्तः the strong? उत्तमौजाः Uttamaujas? च and? वीर्यवान् the brave? सौभद्रः the son of Subhadra? द्रौपदेयाः the sons of Draupadi? च and? सर्वे all? एव even? महारथाः great carwarriors.No Commentary.<|eot_id|>'

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 701 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 22
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.932200
2,1.856700
3,1.813900
4,1.915000
5,1.833000
6,1.787700
7,1.556100
8,1.593100
9,1.557000
10,1.407900


In [13]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [14]:
!zip -r lora_model.zip lora_model/

  adding: lora_model/ (stored 0%)
  adding: lora_model/adapter_config.json (deflated 55%)
  adding: lora_model/tokenizer.json (deflated 85%)
  adding: lora_model/README.md (deflated 66%)
  adding: lora_model/tokenizer_config.json (deflated 94%)
  adding: lora_model/special_tokens_map.json (deflated 71%)
  adding: lora_model/adapter_model.safetensors (deflated 8%)


In [15]:
from google.colab import files
files.download("lora_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
   {'content': 'Shloka: युधामन्युश्च विक्रान्त उत्तमौजाश्च वीर्यवान् |\nसौभद्रो द्रौपदेयाश्च सर्व एव महारथाः ||१-६|| Transliteration: yudhāmanyuśca vikrānta uttamaujāśca vīryavān .\nsaubhadro draupadeyāśca sarva eva mahārathāḥ ||1-6||',
'role': 'user'},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 512,
                   use_cache = True, temperature = 1.5, min_p= 0.1)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


English Meaning: 1.6 And (there is) Yudhamanyu also, of very bold mien, and Utata the brave, and Saubhadra, and the sons of Draupadi, all were maharathis. Hindi Meaning: ।।1.6।। द्रोणसेनाध्यक्ष उत्तमौजाश्च (युधामन्यु) वीर्यवान् सौभद्रो द्रौपदेयाश्च युधामन्यु महरथी सब थे।। Word Meaning: 1.6 युधामन्युः and Yudhamanyu? विक्रान्तः very bold? उत्तमौजाश्च of very bold mien? वीर्यवान् of great courage? सौभद्रः Saubhadra? द्रुपदेयाः sons of Draupadi? च and? सर्वः all? एव even? महारथाः maharathis.Commentary Utamaujashcha very bold. Utama is another name for Dhrishtaadyumna? the son of Drupada. The five sons of Draupadi were Dhristadyumna? Shatrujit? Niramitras and Shatayunj.<|eot_id|>


<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nShloka: युधामन्युश्च विक्रान्त उत्तमौजाश्च वीर्यवान् |\nसौभद्रो द्रौपदेयाश्च सर्व एव महारथाः ||१-६|| Transliteration: yudhāmanyuśca vikrānta uttamaujāśca vīryavān.\nsaubhadro draupadeyāśca sarva eva mahārathāḥ ||1-6||<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nEnglish Meaning: 1.6. "The strong Yodhamanyu and the brave Uttamaujas, the son\nof Subhadra (Abhimanyu, the son of Subhadra and Arjuna), and the sons of\nDraupadi, all of great chariots (great heroes). Hindi Meaning: ।।1.6।।पराक्रमी युधामन्यु,\xa0 बलवान् उत्तमौजा,\xa0 सुभद्रापुत्र (अभिमन्यु) और द्रोपदी के पुत्र -- ये सब महारथी हैं। Word Meaning: 1.6 युधामन्युः Yudhamanyu? च and? विक्रान्तः the strong? उत्तमौजाः Uttamaujas? च and? वीर्यवान् the brave? सौभद्रः the son of Subhadra? द्रौपदेयाः the sons of Draupadi? च and? सर्वे all? एव